In [1]:
from libraries import *
from parameters import *
import statsmodels.api as sm
import sklearn.feature_selection
import statsmodels.api as sm
import sklearn.feature_selection
from sklearn.preprocessing import PolynomialFeatures
import warnings
warnings.filterwarnings('ignore')
from statannot import add_stat_annotation
from numpy.random import seed
from numpy.random import randint
import statsmodels.formula.api as smf
import random

/home/beraslan/miniconda3/lib/python3.8/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
os.getcwd()
os.chdir(projectDir)

In [3]:
adataC = sc.read('outputs/anndata/adata-hash-features_singlets_SingleKO_08122020_PerGENE.h5ad')


In [4]:
adataC.X.shape

(242938, 6685)

In [5]:
adataControl = adataC[adataC.obs['GENE_CONTROL_'] == 1,]

In [6]:
adataControl.X.shape

(41123, 6685)

In [11]:
adataControl.X.shape[0]

41123

In [9]:
random.sample(range(0, adataControl.X.shape[0]), 200)

[30384,
 18030,
 12679,
 38721,
 996,
 3817,
 34079,
 14644,
 38327,
 30051,
 2157,
 23763,
 31379,
 6643,
 22589,
 35204,
 10846,
 22163,
 4219,
 30881,
 8719,
 36317,
 22021,
 19644,
 4676,
 32191,
 16291,
 36418,
 10577,
 7577,
 8639,
 11713,
 24412,
 16566,
 24707,
 9416,
 4947,
 18375,
 4615,
 19467,
 27663,
 4387,
 40022,
 20249,
 6449,
 32070,
 36744,
 36544,
 31847,
 19727,
 40333,
 5721,
 18423,
 36351,
 3230,
 33592,
 11297,
 18385,
 17757,
 15454,
 29225,
 5086,
 1463,
 29782,
 27436,
 34389,
 6232,
 23952,
 2759,
 12360,
 31892,
 4290,
 18198,
 17341,
 39065,
 6764,
 12055,
 376,
 11575,
 22596,
 14752,
 593,
 1306,
 28778,
 28787,
 2578,
 25269,
 30827,
 9143,
 4855,
 4355,
 36132,
 1006,
 26059,
 14697,
 4097,
 39327,
 34732,
 9989,
 18805,
 38681,
 8795,
 24330,
 37939,
 7024,
 12247,
 15978,
 34505,
 32062,
 21956,
 34497,
 11964,
 33143,
 5837,
 394,
 18694,
 5079,
 35813,
 4945,
 3621,
 33103,
 21122,
 22727,
 10092,
 32739,
 38054,
 15585,
 20886,
 22028,
 36238,
 40

In [7]:
randIndex = adataControl.obs.index[random.sample(range(0, adataControl.X.shape[0]), 200)]

In [8]:
randIndex

Index(['AGTCATGTCTCCAATT-1-E3-2', 'AGTGATCAGACCTCCG-1-E3-20',
       'GGGTGAACAGAACTCT-1-E4-11', 'GTGAGGAGTGCTTCAA-1-E3-14',
       'GTAGGAGCAAACCATC-1-E3-1', 'GACTCTCGTTGCACGC-1-E4-9',
       'ACCTGTCCATCAGCTA-1-E3-11', 'TCTGGCTTCCATCACC-1-E3-22',
       'CTCTGGTCACTCCACT-1-E3-5', 'ATCCACCTCTCACTCG-1-E3-9',
       ...
       'AGGAAATCAAATACGA-1-E3-16', 'CTACATTTCGGCTGTG-1-E4-3',
       'CTAACCCGTTAAGTCC-1-E3-20', 'TCGACGGGTCAAACGG-1-E3-16',
       'CTCCTCCCAGCCCACA-1-E4-14', 'CGCATAAGTGACTGAG-1-E3-21',
       'ATCCTATTCTCTCGCA-1-E3-20', 'GAGACTTTCCGAGTGC-1-E4-22',
       'CGTGCTTTCCCATTTA-1-E4-4', 'TGAGCATTCTGTACAG-1-E4-12'],
      dtype='object', length=200)

In [7]:
for i in range(0, 100,1):
    randIndex = adataControl.obs.index[random.sample(range(0, adataControl.X.shape[0]), 200)]
    adataControl.obs["GENE_RandomControl_"+str(i)] = 0
    adataControl.obs.loc[adataControl.obs.index.isin(randIndex),  ("GENE_RandomControl_" + str(i))] = 1 

Trying to set attribute `.obs` of view, copying.


In [8]:
 adataControl.obs

,barcode_rank,channel,empty_drops_FDR,log10_n_genes,log10_n_umis,mt_frac,n_counts,n_genes,n_umis,neg_log10_empty_drops_FDR,...,GENE_RandomControl_90,GENE_RandomControl_91,GENE_RandomControl_92,GENE_RandomControl_93,GENE_RandomControl_94,GENE_RandomControl_95,GENE_RandomControl_96,GENE_RandomControl_97,GENE_RandomControl_98,GENE_RandomControl_99
AAACGCTAGTTGTAGA-1-E3-1,20114.5,1,0.0,3.033021,3.399674,0.086056,2510.0,951,2510.0,4.0,...,0,0,0,0,0,0,0,0,0,0
AAAGGATTCAGACAAA-1-E3-1,16093.0,1,0.0,3.159266,3.533136,0.071198,3413.0,1247,3413.0,4.0,...,0,0,0,0,0,0,0,0,0,0
AAAGGGCAGACGGTTG-1-E3-1,10375.5,1,0.0,3.246499,3.634679,0.073052,4312.0,1514,4312.0,4.0,...,0,0,0,0,0,0,0,0,0,0
AAAGGGCAGAGATCGC-1-E3-1,7871.5,1,0.0,3.258158,3.683587,0.062370,4826.0,1561,4826.0,4.0,...,0,0,0,0,0,0,0,0,0,0
AAAGGGCGTTGATGTC-1-E3-1,17444.5,1,0.0,3.100371,3.501470,0.051056,3173.0,1095,3173.0,4.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGATCCAAACGAGC-1-E4-24,9083.5,24,0.0,3.417306,3.935205,0.049454,8614.0,2248,8614.0,4.0,...,0,0,0,0,0,0,0,0,0,0
TTTGATCCATGACAAA-1-E4-24,7273.5,24,0.0,3.439964,3.972851,0.088248,9394.0,2293,9394.0,4.0,...,0,0,0,0,0,0,0,0,0,0
TTTGATCTCAGCTGTA-1-E4-24,15081.5,24,0.0,3.332842,3.812178,0.060718,6489.0,1839,6489.0,4.0,...,0,0,0,0,0,0,0,0,0,0
TTTGGAGTCCGTAGGC-1-E4-24,5626.5,24,0.0,3.428459,4.013511,0.074738,10316.0,2252,10316.0,4.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
adataControl.obs["GENE_RandomControl_1"].value_counts()

0    40923
1      200
Name: GENE_RandomControl_1, dtype: int64

In [10]:
geneCol = ["GENE_RandomControl_"+str(x) for x in range(0,100,1)]

In [11]:
geneCol = geneCol + ["n_genes", "mt_frac", "leiden" ]

In [12]:
## Add covariates that should be corrected for
guideMatrix = adataControl.obs[geneCol]
guideMatrix.shape

(41123, 103)

In [13]:
expressionMatrix = adataControl.X

In [14]:
expressionMatrix.shape

(41123, 6685)

In [15]:
allVars = pd.Series(geneCol) 
allVars

0       GENE_RandomControl_0
1       GENE_RandomControl_1
2       GENE_RandomControl_2
3       GENE_RandomControl_3
4       GENE_RandomControl_4
               ...          
98     GENE_RandomControl_98
99     GENE_RandomControl_99
100                  n_genes
101                  mt_frac
102                   leiden
Length: 103, dtype: object

In [16]:
all_columns = "+".join(allVars[~allVars.isin(["leiden"])])


In [17]:
my_formula = "y~" + all_columns
my_formula

'y~GENE_RandomControl_0+GENE_RandomControl_1+GENE_RandomControl_2+GENE_RandomControl_3+GENE_RandomControl_4+GENE_RandomControl_5+GENE_RandomControl_6+GENE_RandomControl_7+GENE_RandomControl_8+GENE_RandomControl_9+GENE_RandomControl_10+GENE_RandomControl_11+GENE_RandomControl_12+GENE_RandomControl_13+GENE_RandomControl_14+GENE_RandomControl_15+GENE_RandomControl_16+GENE_RandomControl_17+GENE_RandomControl_18+GENE_RandomControl_19+GENE_RandomControl_20+GENE_RandomControl_21+GENE_RandomControl_22+GENE_RandomControl_23+GENE_RandomControl_24+GENE_RandomControl_25+GENE_RandomControl_26+GENE_RandomControl_27+GENE_RandomControl_28+GENE_RandomControl_29+GENE_RandomControl_30+GENE_RandomControl_31+GENE_RandomControl_32+GENE_RandomControl_33+GENE_RandomControl_34+GENE_RandomControl_35+GENE_RandomControl_36+GENE_RandomControl_37+GENE_RandomControl_38+GENE_RandomControl_39+GENE_RandomControl_40+GENE_RandomControl_41+GENE_RandomControl_42+GENE_RandomControl_43+GENE_RandomControl_44+GENE_RandomContro

In [ ]:
for i in range(0,6680,80):
    print(i)
    pvalDF = pd.DataFrame()
    coefDF = pd.DataFrame()
    
    for j in range(i,i+80):
        print(j)
        guideMatrix["y"] = np.array(expressionMatrix)[:,j]
        est = smf.mixedlm(my_formula, guideMatrix, groups=guideMatrix["leiden"], missing='drop')
        mdf = est.fit(method=["bfgs"])
        pvalDF[str(j)] = mdf.summary().tables[1]["P>|z|"]
        coefDF[str(j)] = mdf.summary().tables[1]["Coef."]
    
    coefDF.index = mdf.summary().tables[1].index
    pvalDF.index = mdf.summary().tables[1].index
    pvalDF.to_csv("pvalues_"+str(i)+"_"+str(i+80)+"_Control_MixedEffect_100_1.csv")   
    coefDF.to_csv("coefs_"+str(i)+"_"+str(i+80)+"_Control_MixedEffec_100_1.csv")   

0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
80
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
270
271
272
273
274
275
276
277